In [1]:
print("hello")

hello


In [6]:
from langchain_community.document_loaders import PyPDFLoader
doc_path="H:\\Github\\GenAI\\RAG\\RAG using Hybrid Search\data\\2005.11401v4.pdf"
loader=PyPDFLoader(doc_path)
docs=loader.load()

<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
C:\Users\HP\AppData\Local\Temp\ipykernel_16144\730254181.py:2: SyntaxWarning: invalid escape sequence '\d'
  doc_path="H:\\Github\\GenAI\\RAG\\RAG using Hybrid Search\data\\2005.11401v4.pdf"


In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(chunk_size=200,chunk_overlap=30)
chunks = splitter.split_documents(docs)

In [29]:
from langchain_community.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
print(embeddings.embed_query("test"))  # should return a list of floats


C:\Users\HP\AppData\Local\Temp\ipykernel_16144\3230838103.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
h:\Github\GenAI\.genaienv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[0.01157346274703741, 0.0251361932605505, -0.03670189157128334, 0.059324879199266434, -0.0071490490809082985, -0.04119426757097244, 0.07708740234375, 0.03744257986545563, 0.012449024245142937, -0.00611764844506979, 0.017034266144037247, -0.07701534777879715, -0.00039417133666574955, 0.027909060940146446, -0.015989137813448906, -0.06827525049448013, 0.008884693495929241, -0.020280746743083, -0.08035992830991745, -0.013074045069515705, -0.04109999164938927, -0.02589806169271469, -0.026538675650954247, 0.03305229917168617, -0.022079166024923325, 0.021046113222837448, -0.05792200192809105, 0.03294878453016281, 0.02970738708972931, -0.06224842742085457, 0.03878801316022873, 0.031990695744752884, 0.015330800786614418, 0.0453069694340229, 0.05314943194389343, 0.013360688462853432, 0.04122495651245117, 0.028142889961600304, 0.019398432224988937, -0.003252344438806176, -0.0036123283207416534, -0.1428602635860443, 0.03807120397686958, -0.010916150175035, 0.026094019412994385, 0.04136991873383522

In [30]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [31]:
# Check embeddings directly
print(embeddings.embed_query("test sentence"))


[0.0429728627204895, 0.09663482010364532, -0.002129176165908575, 0.07826834917068481, -0.006417416967451572, 0.038000259548425674, 0.09461677074432373, 0.00039393361657857895, -0.054561398923397064, 0.014836559072136879, 0.13571231067180634, -0.07155624777078629, 0.019836803898215294, 0.00460875453427434, 0.029340723529458046, -0.02444269135594368, 0.025567643344402313, -0.03157782182097435, -0.06949746608734131, 0.002447790000587702, 0.040835823863744736, -0.015664512291550636, 0.006574304774403572, 0.04475158452987671, 0.004423543345183134, 0.05280672386288643, -0.0522431954741478, 0.020337261259555817, 0.0758800283074379, -0.02192024700343609, -0.022451752796769142, 0.023846246302127838, 0.009509583003818989, 0.08760558813810349, 0.05164284259080887, -0.005793592426925898, 0.0060132392682135105, 0.002463444834575057, 0.01739945448935032, -0.0020238743163645267, -0.0012834791559726, -0.11802417784929276, 0.06548900157213211, -0.0015912101371213794, 0.022106826305389404, 0.00398674281

In [32]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings,
)
vectorstore_retreiver=vector_store.as_retriever(search_kwargs={"k": 3})
#ectorstore_retreiver = vectorstore.as_retriever(search_kwargs={"k": 3})

In [33]:

from langchain.retrievers import BM25Retriever, EnsembleRetriever
keyword_retriever = BM25Retriever.from_documents(chunks)
keyword_retriever.k =  3
ensemble_retriever = EnsembleRetriever(retrievers=[vectorstore_retreiver,keyword_retriever],weights=[0.3, 0.7])

In [34]:
#LLM
import os

from dotenv import load_dotenv
load_dotenv
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
llm.invoke("Sing a ballad of LangChain.")


AIMessage(content="(Verse 1)\nIn realms of code, where silicon did gleam,\nThe mighty LLMs, in a waking dream,\nHeld knowledge vast, but in a lonely plight,\nNo hands to grasp, no tools to bring their light.\nThey answered queries, with a fluent grace,\nBut longed for action, beyond their neural space.\n\n(Verse 2)\nA whisper rose, then grew into a roar,\nA call for structure, to open every door.\nFrom minds inventive, a new vision spun,\nTo chain their brilliance, till tasks were truly done.\nLangChain its name, a promise in its sound,\nTo link the models, on hallowed digital ground.\n\n(Verse 3)\nNo longer solo, in their silent thought,\nBut step by step, a mighty chain was wrought.\nFrom prompt to parse, from query to reply,\nEach link a task, beneath the digital sky.\nA sequence clear, where logic could extend,\nFrom input's start, unto the journey's end.\n\n(Verse 4)\nThen came the Agents, with a clever mind,\nTo choose the pathway, the right tool to find.\nA calculator's logic, a

In [35]:
from langchain.chains import RetrievalQA
normal_chain = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=vectorstore_retreiver
)

In [36]:
hybrid_chain = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=ensemble_retriever
)

In [ ]:

response1 = normal_chain.invoke({"query": "what does this document tells about the Retrieval-Augmented Generation fo Knowledge-Intensive NLP Tasks?"})
print(response1)



{'query': 'what does this document tells about the Retrieval-Augmented Generation forKnowledge-Intensive NLP Tasks?', 'result': 'This document describes Retrieval-Augmented Generation (RAG) for knowledge-intensive NLP tasks as follows:\n\n*   **What it is:** A model that combines an information retrieval component with a sequence-to-sequence generation model.\n*   **How it works:** The retrieval component fetches relevant documents from a large corpus, which are then used by the generation component to produce more accurate and informative responses.\n*   **Purpose/Benefits:** It allows the model to leverage external knowledge beyond its internal parameters, leading to improved performance on tasks requiring up-to-date or specific factual information. It also helps mitigate issues like hallucination and outdated information common in purely generative models.\n*   **Applications:** Examples include open-domain question answering, fact verification, and dialogue generation.\n*   **Chall

In [40]:
response1


{'query': 'what does this document tells about the Retrieval-Augmented Generation forKnowledge-Intensive NLP Tasks?',
 'result': 'This document describes Retrieval-Augmented Generation (RAG) for knowledge-intensive NLP tasks as follows:\n\n*   **What it is:** A model that combines an information retrieval component with a sequence-to-sequence generation model.\n*   **How it works:** The retrieval component fetches relevant documents from a large corpus, which are then used by the generation component to produce more accurate and informative responses.\n*   **Purpose/Benefits:** It allows the model to leverage external knowledge beyond its internal parameters, leading to improved performance on tasks requiring up-to-date or specific factual information. It also helps mitigate issues like hallucination and outdated information common in purely generative models.\n*   **Applications:** Examples include open-domain question answering, fact verification, and dialogue generation.\n*   **Chal

In [ ]:
print(response1.get("result"))

In [42]:
response2 = hybrid_chain.invoke("What is Abstractive Question Answering?")

In [43]:
response2

{'query': 'What is Abstractive Question Answering?',
 'result': 'Abstractive Question Answering is a type of question answering where models can generate free-form, abstractive answers. It goes beyond simple extractive QA and can provide answers even when the correct answer is not explicitly present in any retrieved document. For instance, an abstractive model achieved 11.8% accuracy in such cases for NQ, whereas an extractive model would score 0%.'}

In [44]:
print(response2.get("result"))

Abstractive Question Answering is a type of question answering where models can generate free-form, abstractive answers. It goes beyond simple extractive QA and can provide answers even when the correct answer is not explicitly present in any retrieved document. For instance, an abstractive model achieved 11.8% accuracy in such cases for NQ, whereas an extractive model would score 0%.
